In [ ]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import gc
from scipy.stats.mstats import mode
import scipy


# Idea
Caracterizar a usuarios y avisos mediante sus propios atributos y relaciones entre ellos.
Se tomaran en cuenta para este notebook.

- Usuarios
    - Propios:
        - sexo: 1 hombre, 0 mujer y otros.
    - De avisos:
        - Proporcion de vistas sobre las areas mas postuladas
- Avisos
    - Propios
        - Area dentro de las mas postuladas
    - De usuarios:
        - Proporcion de hombres/mujeres que se postulan al aviso
        

# 1. Usuarios

## 1.1 Genero 

In [ ]:
genyedad = pd.concat([
    pd.read_csv("../input/datos/fiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_2_postulantes_genero_y_edad.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_2_postulantes_genero_y_edad.csv")
])
genyedad.head()

In [ ]:
genyedad.drop_duplicates(subset=['idpostulante'], inplace=True)

In [ ]:
genyedad.isnull().sum()

In [ ]:
genyedad['fechanacimiento'] = pd.to_datetime(genyedad['fechanacimiento'], errors='coerce')
genyedad['fechanacimiento'] = 2018 - genyedad['fechanacimiento'].dt.year
genyedad.rename(columns={'fechanacimiento':'edad'}, inplace=True)
genyedad.head()

In [ ]:
genyedad.isnull().sum()

In [ ]:
genyedad['edad'].fillna(genyedad['edad'].mean(), inplace=True)
genyedad.isnull().sum()

In [ ]:
# Sexo
# 1 MASC
# 0 FEM y otro
genyedad['sexo'] = genyedad['sexo'].apply(lambda x: 1 if x =='MASC' else 0)
genyedad = genyedad.astype({'sexo':'uint8'})
genyedad.head()


In [ ]:
gc.collect()

In [ ]:
# Por convencion llamo a usuarios a los datos que caracterizan a los postulantes con sus propios datos
usuarios = genyedad
usuarios.head()

In [ ]:
usuarios.isnull().sum()

# 1.1 Educacion

In [ ]:
educacion = pd.concat([
    pd.read_csv("../input/datos/fiuba_1_postulantes_educacion.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_1_postulantes_educacion.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_1_postulantes_educacion.csv")
])
educacion.head()

In [ ]:
educacion.info()

In [ ]:
educacion.isnull().sum()

In [ ]:
# Nos quedamos con los graduados
educacion = educacion[educacion['estado'] == 'Graduado']
educacion.drop(columns='estado',inplace=True)
educacion.head()

In [ ]:
educacion.drop_duplicates(subset=['idpostulante','nombre'], inplace=True)

In [ ]:
educacion['nombre'].value_counts()

In [ ]:
# Anios de formacion academica despues de la secundaria (aprox)
formacion = {
    'Secundario':0,
    'Terciario/Técnico':3,
    'Universitario':5,
    'Posgrado':6,
    'Master':7,
    'Doctorado':10
}

In [ ]:
educacion['nombre'] = educacion['nombre'].apply(lambda x: formacion[x] if x in formacion else 0)
educacion.head()

In [ ]:
educacion.rename(columns={'nombre':'formacion'}, inplace=True)

In [ ]:
educacion['formacion'].mean()

In [ ]:
# caracterizo al usuarios por su mayor nivel academico alcanzado
educacion = educacion.pivot_table(index='idpostulante', aggfunc='max')
educacion.reset_index(inplace=True)
educacion.head()

# 1.2 Caracterizacion de usuarios

In [ ]:
genyedad.isnull().sum()

In [ ]:
educacion.isnull().sum()

In [ ]:
usuarios = pd.merge(genyedad, educacion, on='idpostulante', how='outer')
usuarios.head()

In [ ]:
usuarios.isnull().sum()

In [ ]:
# relleno los anios de formacion academica con el promedio.
usuarios['formacion'] = usuarios['formacion'].fillna(usuarios['formacion'].mean())
usuarios.isnull().sum()

In [ ]:
usuarios.head()

# 2. Areas mas postuladas


In [ ]:
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

In [ ]:
postulaciones.isnull().sum()

In [ ]:
postulaciones.drop(columns='fechapostulacion', inplace=True)
postulaciones.head()

In [ ]:
postulaciones.drop_duplicates(subset=['idaviso','idpostulante'], inplace=True)

In [ ]:
gc.collect()

## 2.1 Avisos

In [ ]:
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])
avisos.head()

In [ ]:
avisos.drop_duplicates(subset='idaviso', inplace=True)

In [ ]:
# Los tipicos que estan llenos de nul
avisos.drop(columns=['idpais', 'titulo', 'descripcion', 'ciudad', 'mapacalle', 'denominacion_empresa'], inplace=True)
avisos.head()

In [ ]:
gc.collect()

In [ ]:
# Zona
# 1 GBA
# 0 otros
avisos['nombre_zona'] = avisos['nombre_zona'].apply(lambda x: 1 if x == 'Gran Buenos Aires' else 0)

In [ ]:
# Tipo de trabajo
# 9 Full-time
# 6 Part-time
# 4 otros
# El numero estaria representando mayor cantidad de horas semanales de trabajo.
avisos['tipo_de_trabajo'] = avisos['tipo_de_trabajo'].apply(lambda x: 9 if x == 'Full-time' else (6 if x == 'Part-time' else 4))

In [ ]:
# Nivel laboral
# Orden jerarquico
# N: representa el sueldo, al subir de nivel jerarquico deberia duplicarse
nivel = {
    'Gerencia / Alta Gerencia / Dirección': 16,   
    'Jefe / Supervisor / Responsable': 8,        
    'Senior / Semi-Senior': 4,                    
    'Junior': 2,                                   
    'Otro': 1,                                    
}
avisos['nivel_laboral'] = avisos['nivel_laboral'].apply(lambda x: nivel[x] if x in nivel else 0)

In [ ]:
avisos.head()

In [ ]:
avisos.isnull().sum()

## 2.2 Areas mas postuladas de los avisos

In [ ]:
avisos_postulados = pd.merge(postulaciones, avisos,on='idaviso', how='inner')
avisos_postulados.head()

In [ ]:
# Idea: OneHotEncoding esta ocupando demasiada RAM, entonces voy a asignar un numero a cada categoria que tenga sentido.
# Este numero sera su posicion en el ranking de areas mas postulada.
# Por que tiene sentido? Porque aquellas areas mas cercanas al 0 son las mas importantes, mientras que las mas alejadas son menos importantes.
# Es mas probable que se postule a un area de valor bajo (0=Ventas) a uno de valor alto (185=Fonoaudiologia)
areas_mas_postuladas = pd.DataFrame(avisos_postulados['nombre_area'].value_counts()).reset_index().reset_index()
areas_mas_postuladas.head()

In [ ]:
areas_mas_postuladas.rename(columns={'level_0':'ranking', 'nombre_area':'cant_post'}, inplace=True)
areas_mas_postuladas.rename(columns={'index':'nombre_area'}, inplace=True)
areas_mas_postuladas.head()

In [ ]:
dicc = areas_mas_postuladas[['nombre_area']].to_dict()
dicc1 = dicc['nombre_area']
ranking_areas = {v: k for k, v in dicc1.items()}
ranking_areas

# 3. Avisos

In [ ]:
avisos.head()

In [ ]:
# 255 indica que es un area que no aparece en el ranking de postulaciones
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: ranking_areas[x] if x in ranking_areas else 255)
avisos = avisos.astype({'nombre_area':'uint8'})
avisos.head()

# 4. Vistas

In [ ]:
vistas = pd.concat([
    pd.read_csv("../input/datos/fiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_3_vistas.csv")
])
vistas.head()

In [ ]:
# Este es el DataFrame que utiliza mayor cantidad de RAM por lo tanto voy a probar eliminar los duplicados para guardar algo de memoria.
vistas.drop(columns='timestamp', inplace=True)
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)
vistas.head()

In [ ]:
gc.collect()

In [ ]:
vistas.isnull().sum()

# 5. Armado de Datos

In [ ]:
# Voy a armar los datos con info de usuarios y avisos.

In [ ]:
datos = pd.merge(usuarios, postulaciones, on='idpostulante', how='inner')
datos.head()

In [ ]:
datos = pd.merge(datos, avisos, on='idaviso', how='inner')
datos.head()

In [ ]:
# no se porqe hay duplicados pero los voy  a eliminar
datos.drop_duplicates(subset=['idpostulante', 'idaviso'], inplace=True)

In [ ]:
# En todos estos ya sabemos que el usuario se postula.
datos['sepostula'] = 1
datos.head()

In [ ]:
datos = datos.astype({'edad':'int8', 'sexo':'int8', 'nombre_zona':'int8', 'tipo_de_trabajo':'int8', 'nivel_laboral':'int8', 'sepostula':'int8'})

In [ ]:
datos.isnull().sum()

* ## 5.1 Usuarios y sus vistas

In [ ]:
f = lambda x: scipy.stats.mode(x, axis=None, nan_policy='omit')[0][0]
datos.head()

In [ ]:
vistas.head()

In [ ]:
datos['idpostulante'].values

In [ ]:
# Solo me quedo con los usuarios que quiero buscar
vistas_usuarios = vistas[vistas['idpostulante'].isin(datos['idpostulante'].values)]

In [ ]:
vistas_usuarios.head()

In [ ]:
vistas_usuarios = pd.merge(vistas_usuarios, avisos, on='idaviso', how='left')

In [ ]:
# Tomo la moda de nombre_zona, tipo_de_trabajo, nivel_laboral y nombre area del postulante
vistas_usuarios = vistas_usuarios.drop(columns='idaviso').pivot_table(index='idpostulante',
                                                                       aggfunc=f)

In [ ]:
vistas_usuarios.head()

In [ ]:
vistas_usuarios.rename(columns={'nivel_laboral':'lab_fav',
                                'nombre_area':'area_fav',
                                'nombre_zona':'zona_fav',
                                'tipo_de_trabajo':'trab_fav'},
                       inplace=True)
vistas_usuarios.head()

In [ ]:
datos = pd.merge(datos, vistas_usuarios, on='idpostulante', how='left')
datos.head()

In [ ]:
datos.isnull().sum()

In [ ]:
datos['lab_fav'] = datos['lab_fav'].fillna(f(datos['lab_fav']))
datos['area_fav'] = datos['area_fav'].fillna(f(datos['area_fav']))
datos['zona_fav'] = datos['zona_fav'].fillna(f(datos['zona_fav']))
datos['trab_fav'] = datos['trab_fav'].fillna(f(datos['trab_fav']))

In [ ]:
datos.isnull().sum()

# 6. TEMPORAL: Generacion de NO_POSTULA

In [ ]:
datos.size

In [ ]:
datos.head()

In [ ]:
# Usuarios
us_no_post = datos[['idpostulante', 'edad' ,'sexo', 'lab_fav', 'area_fav', 'zona_fav', 'trab_fav']].sample(replace=True, frac=1)
# Avisos
avisos_no_post = datos[['idaviso', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area']]
avisos_no_post['sepostula'] = 0
avisos_no_post = avisos_no_post.sample(replace=True, frac=1)

In [ ]:
us_no_post.reset_index(drop=True, inplace=True)
us_no_post.head() 

In [ ]:
us_no_post.isnull().sum()

In [ ]:
avisos_no_post.reset_index(drop=True, inplace=True)
avisos_no_post.head()

In [ ]:
datos_no_post = pd.concat([us_no_post, avisos_no_post], axis=1)
datos_no_post.head()

In [ ]:
# Reordeno las columnas
datos = datos[['idpostulante', 'edad', 'sexo', 'lab_fav', 'area_fav', 'zona_fav', 'trab_fav', 'idaviso', 'nombre_zona', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'sepostula']]
datos.head()

In [ ]:
#datos.reset_index(drop=True, inplace=True)
#datos_no_post.reset_index(drop=True, inplace=True)

In [ ]:
# agrupo todos en uno
df = pd.concat([datos, datos_no_post])
df.head()

In [ ]:
df['sepostula'].value_counts()

In [ ]:
df.isnull().sum()

# 7. TEMPORAL : Algoritmos de Machine Learning****

In [ ]:
X = df.drop(columns=['idpostulante', 'idaviso', 'sepostula'])

In [ ]:
X.head()

In [ ]:
X.isnull().sum()

In [ ]:
y = df['sepostula']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test) 

In [ ]:
#clf = MLPClassifier(random_state=0)
#clf = KNeighborsClassifier(n_neighbors=5) 
#clf = DecisionTreeClassifier(random_state=0)
#clf = RandomForestClassifier()
clf = Perceptron(n_iter=40, eta0=0.1, random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)  
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

In [ ]:
# Visualizacion de Decision Tree
# converting into the pdf file
#with open("tree.dot", "w") as f:
#    f = tree.export_graphviz(clf, out_file=f, feature_names=X.columns)

# 8. TEMPORAL: Armo los datos para enviar el Summit

In [ ]:
final100k = pd.read_csv("../input/finalfinal/test_final_100k.csv")
final100k.info()

In [ ]:
final100k.isnull().sum()

In [ ]:
# Busco sus datos de educacion
final100k = pd.merge(final100k, usuarios, on='idpostulante', how='left')

In [ ]:
final100k.head()

In [ ]:
final100k.isnull().sum()

In [ ]:
# Busco la informacion de los avisos
final100k = pd.merge(final100k, avisos, on='idaviso', how='left')

In [ ]:
final100k.head()

In [ ]:
final100k.isnull().sum()

# PREDICCION

In [ ]:
y_final = clf.predict(final100k.drop(columns=['id', 'idaviso', 'idpostulante']))

In [ ]:
y_summit = pd.DataFrame(y_final)
y_summit.columns = ['sepostulo']
summit = y_summit.reset_index()
summit.head()
summit.rename(columns={'index':'id'}, inplace=True)
summit.to_csv("summitGSNB.csv", index=False)